# Keras Merge Applied

For complex models use the Functional API. Sequential model cannot handle merges, models with multiple inputs and outputs, models with shared layers.

Reference: https://www.puzzlr.org/the-keras-functional-api-five-simple-examples/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tensorflow.keras import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, concatenate
from tensorflow.keras.utils import plot_model

### Generate Data

In [ ]:
n_row = 1000
x1 = np.random.randn(n_row)
x2 = np.random.randn(n_row)
x3 = np.random.randn(n_row)
y_classifier = np.array([1 if (x1[i] + x2[i] + (x3[i])/3 + np.random.randn(1) > 1) else 0 for i in range(n_row)])
y_cts = x1 + x2 + x3/3 + np.random.randn(n_row)
dat = np.array([x1, x2, x3]).transpose()

In [ ]:
# Plot data
plt.scatter(dat[:,0],dat[:,1], c=y_classifier)
plt.show()

In [ ]:
# Generate indexes of test and train 
idx_list = np.linspace(0,999,num=1000)
idx_test = np.random.choice(n_row, size = 200, replace=False)
idx_train = np.delete(idx_list, idx_test).astype('int')
 
# Split data into test and train
dat_train = dat[idx_train,:]
dat_test = dat[idx_test,:]

y_classifier_train = y_classifier[idx_train]
y_classifier_test = y_classifier[idx_test]
y_cts_train = y_cts[idx_train]
y_cts_test = y_cts[idx_test]

### Logistic Model

Build logistic regression using the Keras functional model. 

The input layer needs to have shape (p,) where p is the number of columns in your training matrix. In our case we have three columns (x_1, x_2, x_3) so we set the shape to (3,)

The output layer needs to have the same number of dimensions as the number of neurons in the dense layer. In our case we’re predicting a binary vector (0 or 1), which has 1 dimension, so our dense layer needs to have one neuron.

In [ ]:
# Build the model with Functional API
inputs = Input(shape=(3,))
output = Dense(1, activation='sigmoid')(inputs)
logistic_model = Model(inputs, output)
 
# Compile the model 
logistic_model.compile(optimizer='sgd',
                       loss = 'binary_crossentropy',
                       metrics=['accuracy'])

In [ ]:
logistic_model.summary()

In [ ]:
# Fit on training data
logistic_model.fit(x=dat_train, y=y_classifier_train, epochs = 500, verbose=0,
                   validation_data = (dat_test, y_classifier_test))
logistic_model.fit(x=dat_train, y=y_classifier_train, epochs = 1, verbose=1,
                   validation_data = (dat_test, y_classifier_test))

In [ ]:
plot_model(logistic_model,show_shapes = True, show_layer_names = True)

### Deep Model

Below we train a neural network with a large number of hidden layers. We also add Dropout to the layers to reduce overfitting.

It can be useful sometimes to use for loops and if statements when using the Functional API, particularly for complicated models.

This model does about as well as the previous neural network. It could probably do better by tuning the hyperparameters, like the amount of dropout or the number of neural network layers.

In [ ]:
# specify how many hidden layers to add (min 1)
n_layers = 5
 
inputs = Input(shape=(3,))
x = Dense(200, activation='relu')(inputs)
x = Dropout(0.4)(x)
for layer in range(n_layers - 1):
    x = Dense(200, activation='relu')(x)
    x = Dropout(0.3)(x)

output = Dense(1, activation='sigmoid')(x)
deep_n_net = Model(inputs, output)
 
deep_n_net.compile(optimizer = 'adam', loss= 'binary_crossentropy', metrics=['accuracy'])
 
deep_n_net.fit(dat_train, y_classifier_train, epochs = 50, verbose=0,
validation_data = (dat_test, y_classifier_test))
deep_n_net.fit(dat_train, y_classifier_train, epochs = 1, verbose=1,
validation_data = (dat_test, y_classifier_test))

In [ ]:
deep_n_net.summary()

In [ ]:
plot_model(deep_n_net,show_shapes = True, show_layer_names = True)

### Simple Neural Network + Metadata

One use case for the Functional API is where you have multiple data sources that you want to pull together into one model.

For example, if your task is image classification you could use the Sequential model to build a convolutional neural network that would run over the images. If you decide to use the Functional API instead of the Sequential model, you can also include metadata of your image into your model: perhaps its size, date created, or its tagged location.

#### Add Metadata

In [ ]:
metadata_1 = y_classifier + np.random.gumbel(scale = 0.6, size = n_row)
metadata_2 = y_classifier - np.random.laplace(scale = 0.5, size = n_row)
metadata = np.array([metadata_1,metadata_2]).T
 
# Create training and test set
metadata_train = metadata[idx_train,:]
metadata_test = metadata[idx_test,:]

### Merge Model

In [ ]:
input_dat = Input(shape=(3,)) # for the three columns of dat_train
n_net_layer = Dense(50, activation='relu') # first dense layer
x1 = n_net_layer(input_dat)
x1 = Dropout(0.5)(x1)
 
input_metadata = Input(shape=(2,))
x2 = Dense(25, activation= 'relu')(input_metadata)
x2 = Dropout(0.3)(x2)
 
con = concatenate(inputs = [x1,x2] ) # merge in metadata
x3 = Dense(50)(con)
x3 = Dropout(0.3)(x3)
output = Dense(1, activation='sigmoid')(x3)
meta_n_net = Model(inputs=[input_dat, input_metadata], outputs=output)
 
meta_n_net.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
plot_model(meta_n_net,show_shapes = True, show_layer_names = True)

In [ ]:
meta_n_net.fit(x=[dat_train, metadata_train], y=y_classifier_train, epochs=50, verbose=0,
validation_data=([dat_test, metadata_test], y_classifier_test))
meta_n_net.fit(x=[dat_train, metadata_train], y=y_classifier_train, epochs=1, verbose=1,
validation_data=([dat_test, metadata_test], y_classifier_test))